In [3]:
package main
import (
    "fmt"      // format, writing to devices
    "strconv"  // String conversion utilities
)

Fmt deals with formatting utilities while strconv is, as its name implies, used for converting types to strings.

In [ ]:
func convert() {
    var i int = 42
    fmt.Printf("%v, %T\n", i, i)
    
    // convert to a string
    var s string = strconv.Itoa(i)
    fmt.Printf("%v, %T\n", s, s)
}

convert()

Some things I find different:
 - Capitalizing variables leads to them being exported.
 - Rune strings, don't know much about them.
 - `iota` a weird constant scoped enumeration counter.
 
### Arrays/Slices
 
Array declaring/initializing is so different than what I'm used to. For example, a 2d int array:

In [5]:
func arr2d(){
    var matrix [3][3]int
    matrix[0] = [3]int{1, 1, 1}
    matrix[1] = [3]int{1, 0, 1}
    matrix[2] = [3]int{0, 1, 0}

    // printing is easy, though.
    fmt.Println("Matrix: ", matrix)
}

arr2d()

Matrix:  [[1 1 1] [1 0 1] [0 1 0]]


Good thing is array variables don't behave as they do in Python for example. Assigning a matrix from one variable to another and then changing the array via the initial variable doesn't affect the second. A deep-copy is apparently performed during assignment:

In [8]:
func assign_arr(){
    var a [3]int = [3]int{1, 2, 3}
    b := a
    a[2] = 10
    fmt.Println(a)
    fmt.Println(b)
}

assign_arr()

[1 2 10]
[1 2 3]


Though we *can* create references to it via the common address-of (`&`) characted.

Slices are initialized slightly differently; plus, they don't generally appear to own the underlying data, they behave more like references to it. As a result, assignment just copies the reference and mutation to any of the variable pointing to the slice is reflected in all others:

In [12]:
func assign_slice(){
    var a []int = []int{1, 2, 3}
    b := a
    a[2] = 10
    fmt.Println(a)
    fmt.Println(b)
}

assign_slice()

[1 2 10]
[1 2 10]


Slicing an array/slice is supported and looks exactly the same as in Python (i.e slicing a full array is `[:]`):

In [16]:
func slices(){
    var a [5]int = [5]int{0, 1, 2, 3, 4}
    var b []int = []int{0, 1, 2, 3}
    c := a[:]
    d := b[1:2]
    
    fmt.Println(c)
    // Alteration shouldn't appear in a (it does!)
    c[3] = 0
    fmt.Println(a, c)
    // Alteration should appear in b
    d[0] = 20
    fmt.Println(b, d)
}

slices()

[0 1 2 3 4]
[0 1 2 0 4] [0 1 2 0 4]
[0 20 2 3] [20]


Arrays are static collections, that is, their size does not change during execution of the program. Slices on the other hand are dynamic, we can use the `append` built in function to add elements to the end of the slice:

In [21]:
func append_slice(){
    a := []int{}
    fmt.Println(a)
    fmt.Printf("Lenght: %v\n", len(a))
    fmt.Printf("Capacity: %v\n", cap(a))
    a = append(a, 1)
    fmt.Println(a)
    fmt.Printf("Lenght: %v\n", len(a))
    fmt.Printf("Capacity: %v\n", cap(a))
}

append_slice()

[]
Lenght: 0
Capacity: 0
[1]
Lenght: 0
Capacity: 0


The relationship between the new slice returned and the slice passed in as first argument are actually not as simple as I initially thought. Needs more examination.

### Control flow:

Common constructs:

 - `if` statements
 - `switch` statements (though `type` cases are a nice addition
 

 ### `defer`, `panic`, `recover`

Error handling facilities (probably the tip of the iceberg?). As in Rust, we do not have exceptions.

#### `panic`:

Use it in situations where we can't really continue. `defer` statements/functions will still be executed at the end.

#### `recover`

Only really found inside `defer` functions in order to handle a panic. `recover` seems to be like `sys.exc_info` in Python. It lets you get a handle on a previously invoked `panic` and evaluate whether you can handle it or pass it up along the call stack.

### Pointers

Not different than pointers as seen in `C` so far. Declared with `*` before the type. `&` is, as with `C`, the address operator. Dereferencing the pointer is, again, done with `*`.

Pointer arithmitic isn't defined in normal operation. Apparently a package called `unsafe` is available that allows us to use operations that bypass the type safety of the language.

A `nil` pointer exists so pointer exceptions are a thing. `new` is used to create a new empty reference to a given type, apparently.  

### Functions

`main` is a function which also works as the entry point into our program. Naming functions is similar to naming for other types.

Parameters are positional and passed by value (of course, passing a pointer basically results in pass by reference).
Parameter can also be variadic (`param ...type`). Similar to `*args` in Python.

Return value is placed after the closing parentheses for the parameters and before the opening curly brace starting the function body. Fun fact, returning a pointer to a local variable in a function doesn't result in reading junk (from the deallocated from of the function). Instead, Go realizes we return a pointer to a local and moves the value for that variable into shared memory. (Thereby promoting the lifetime of that value for the duration of the program execution?)

Named return values are also interesting. We specify the name of the local variable to be returned and then add a bare `return`. Go finds the local variable named as a return and substitutes it.

Multiple returns are commonly used to return a result and a `err` (of type `error`) value denoting if the result is valid.

Functions are first class objects. They can be defined in local scope, assigned to values and passed around as any other object. 

Methods are also defined in a different way. Instead of defining them in a different block/namespace we simply provide information on what this function should operate by a separate context-like clause before the function name. This context-clause can choose to accept the instance of a given type either via value or via reference.

### Interfaces

